In [1]:
import numpy as np
import matplotlib.pyplot as plt
import glob
from scipy.optimize import curve_fit

# Author: Josipa
# Updated: 2025-01-25

# To calculate R from D, Tau d, and known r_0
def D_from_r0_and_Td(r_0, T_d):
    return (r_0 **2 )/(4*T_d)

def R_from_D(D, n=0.93*10**(-3), T=298):
    kB = 1.38 * 10**(-23)
    return (kB*T)/(6*np.pi*n*D)

# From known sample, found
r_0_known = 0.766*10**(-6)  # um to m

# function to convert fit to R
def R_from_Td(T_d, viscosity=0.93*10**(-3)):
    return R_from_D(D_from_r0_and_Td(r_0_known, T_d), viscosity)

## Starting Point
Below is the code copied from your analysis so that it only makes one plot. It can then be simplified and text made larger. I cant run it as I dont have the data downloaded before my class. Also I wanted it to be copy-paste able into your FCS analysis notebook so the location of the file should match.

In [ ]:
# Load data
filename = 'Lab 1 - FCS/data/Jan23_600fold_data.lvm'  #! Change this if you want a different file

# Load data
tau, correlation, error = np.genfromtxt(filename, skip_header=23, unpack=True)

# Optional: Remove first N seconds
min_tau = 1*10**(-3)
correlation = correlation[np.where(tau > min_tau )]
error = error[np.where(tau > min_tau )]
tau = tau[np.where(tau > min_tau )]

# Curve fitting
fig, (ax_plot, ax_residual) = plt.subplots(2, 1, figsize=(8, 6), gridspec_kw={'height_ratios': [3, 1]})
# Do best fit
print("\nNot weighted fit")
popt, pcov = curve_fit(G, tau, correlation, p0=[50, 0.9], absolute_sigma=True, 
                        bounds=([0, 0], [np.inf, np.inf]))
fit_error = np.sqrt(np.diag(pcov))
# Print  fit results
print(filename)
print(f'Neff = {popt[0]:.1f} ± {fit_error[0]:.1f}')
print(f'tau_d = {popt[1]:.4f} ± {fit_error[1]:.4f} [s]')
# Calculate residuals and chi squared
residuals = G(tau, *popt) - correlation
norm_residuals = residuals / error
chi_sq = np.sum(norm_residuals**2)
print('chi squared =',chi_sq)
nu=len(tau)-len(popt)
print('degrees of freedom =', nu)
# plot fit
ax_plot.plot(tau, G(tau, *popt), '-', label='unweighted fit', color='blue')
ax_residual.plot(tau, norm_residuals, '.', color='blue', alpha=0.7)

# caolculate what R should be for this data, using weighted fit
R_fit = R_from_Td(popt[1])*10**9
print(f'R = {R_fit} [nm]')

# Do best fit
print("\nWeighted fit")
popt, pcov = curve_fit(G, tau, correlation, p0=[50, 0.9], sigma=error, absolute_sigma=True,
                        bounds=([0, 0], [np.inf, np.inf]))
fit_error = np.sqrt(np.diag(pcov))
# Print  fit results
print(filename)
print(f'Neff = {popt[0]:.1f} ± {fit_error[0]:.1f}')
print(f'tau_d = {popt[1]:.4f} ± {fit_error[1]:.4f} [s]')
# Calculate residuals and chi squared
residuals = G(tau, *popt) - correlation
norm_residuals = residuals / error
chi_sq = np.sum(norm_residuals**2)
print('chi squared =',chi_sq)
nu=len(tau)-len(popt)
print('degrees of freedom =', nu)
# plot fit
ax_plot.plot(tau, G(tau, *popt), '-', label='weighted fit', color='green')
ax_residual.plot(tau, norm_residuals, '*', color='green', alpha=0.7)

# Plot data and fit
# plot data
ax_plot.errorbar(tau, correlation, yerr=error, fmt='.-', label='data', color='black', capsize=3)
# Format plot
ax_residual.set_xlabel('$\\tau$ (s)')
ax_plot.set_ylabel('$G_{\\tau}$')
ax_plot.set_xscale('log')
ax_plot.set_title(filename.split('/')[-1])
ax_plot.legend()
# Format residuals
ax_residual.axhline(0, color='grey', linestyle='--', zorder=1)
ax_residual.axhline(1, color='lightgrey', linestyle='--', zorder=1)
ax_residual.axhline(-1, color='lightgrey', linestyle='--', zorder=1)
ax_residual.set_ylabel('Normalized\nResiduals')
plt.savefig(f'Lab 1 - FCS/plots/{filename.split("/")[-1][:-4]}.png', dpi=200)
plt.show()